scirpt written by DE to generate import tabs from scoring templates and formatted to be imported into redcap

#### import packages

In [1]:
import os
import pandas as pd
import sys
import glob
import re
import numpy as np
import openpyxl as px
import warnings
with warnings.catch_warnings(record=False):
    warnings.simplefilter("always")

install required packages (if needed)

In [33]:
%%bash
python -V

Python 3.9.18


install required packages (if needed)

In [41]:
def install(name,ver):
    import subprocess
    subprocess.call(['pip', 'install',name+'=='+ver])
    
install('xlrd','2.0.0')


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
all_assess_dirs='/Volumes/de_encryptd/np/2024_06_11'
# all_assess_dirs='/Users/daelsaid/scratch/Neuropsych/2023_11_06/updated_scores/'
#/Users/daelsaid/scratch/NeuroPsych/all_assessments_box_final/'
# /Users/daelsaid/scratch/np/behavioral_data-selected_2/all_neuropsych_pc-all'
# /Users/daelsaid/scratch/np/behavioral_data-selected/all_from_main_box
os.chdir(all_assess_dirs)

os.listdir()


['Handedness_scoring_template_9054_1.xlsx',
 'MET_WISC_V_9072_1.xlsx',
 'MET_neuropsych_CBO_9302_1.xlsx',
 'MET_Pre_Strategy_Assessment_9072_1.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9316_1.xlsx',
 'MET_WASI_II_9072_1.xlsx',
 'MET_ATM_9128_2.xlsx',
 'MET_MARS(SEMA)_9072_1.xlsx',
 'DKEFS_MET_9054_1.xlsx',
 'MET_WIAT_III_9228_1.xlsx',
 'MET_Learning_Curve_9476_1.xlsx',
 'MET_Pre_Strategy_Assessment_9156_1.xlsx',
 'MET_WISC_V_9156_1.xlsx',
 'MET_ATL_9501_1.xlsx',
 'MET_WIAT_III_9181_1.xlsx',
 'MET_WASI_II_9156_1.xlsx',
 "MET_Geary's_9857_2.xlsx",
 'ATM_MET_9054_1.xlsx',
 'MET_ATL_9188_1.xlsx',
 'MET_SPT_CBO_9476_1.xlsx',
 'MET_MARS(SEMA)_9156_1.xlsx',
 'MET_WIAT_III_9347_1.xlsx',
 'MET_Subtraction_9501_1.xlsx',
 'MET_MARS(SEMA)_9228_1.xlsx',
 'MET_WISC_V_9347_1.xlsx',
 'MET_Handedness_9128_1.xlsx',
 'MET_WJ_III_9054_1.xlsx',
 'MET_Pre_Strategy_Assessment_9347_1.xlsx',
 'MET_Learning_Curve_9128_1.xlsx',
 'MET_MARS(SEMA)_9181_1.xlsx',
 'MET_DKEFS_9194_1.xlsx',
 'MET_WASI_II_9347_1.xls

##### Handedness

###### Score handedness data

In [4]:
handedness_scores={'Both':0.5,'Right hand':1,'Left Hand':1,'Right sometimes other':0.5,'Left sometimes other':'0.5'}

In [5]:
# all_assess_dirs='/Users/daelsaid/scratch/Neuropsych/20230914/asx'
# os.chdir(all_assess_dirs)

subj_with_handedness=[]
for subj_data_file in os.listdir(all_assess_dirs):
    if 'andedn' in subj_data_file and subj_data_file.endswith('.xlsx') and '._' not in subj_data_file:
        # print(subj_data_file)
    # if 'andedn' in subj_data_file and subj_data_file.startswith('') and subj_data_file.endswith('.xlsx'):
    # if 'andedn' in subj_data_file: 
        file_prefix=subj_data_file.split('.')[0]
        subj_df1=pd.read_excel(subj_data_file,header=[0],sheet_name=None,skiprows=None,dtype=str)
        keys=subj_df1.keys()
        if len(subj_df1.keys())>=4:
            k=tuple(subj_df1.keys())[-4]
            subj_df=pd.read_excel(subj_data_file,sheet_name=k,header=[0],skiprows=None,dtype=str)
            try:
                for i in range(19, len(subj_df)):
                    handedness_value = subj_df.at[i, 'Unnamed: 1'][0:]
                    
                for i in range(18,19):
                    lq_score = subj_df.at[i, 'Unnamed: 1'][0:]
                    print(file_prefix,float(lq_score),handedness_value.replace(' ','_'))
            except:
                continue
                # lq_score=subj_df.iloc[18:,1]
            # handedness=subj_df.iloc[19:,1]
            # print(file_prefix,lq_score,handedness)
        if len(subj_df1.keys())<=3:
            right=[]
            left=[]
            k=tuple(subj_df1.keys())[-2]
            subj_df=pd.read_excel(subj_data_file,sheet_name=k,header=[0],skiprows=None,dtype=str)
            for num in range(1,len(subj_df.iloc[-1,:])):
                resp=subj_df.iloc[0,num]
                if resp=='Right hand':
                    right.append(1)
                    left.append(0)
                if resp=='Left hand':
                    right.append(0)
                    left.append(1)
                if resp=='Both':
                    right.append(0.5)
                    left.append(0.5)
                if resp=='Right sometimes other':
                    right.append(0.5)
                    left.append(0)
                if resp=='Left sometimes other':
                    right.append(0)                    
                    left.append(0.5)
            try:
                LQ_score=((sum(right)-sum(left))/(sum(right)+sum(left))*100)
                if LQ_score < -40:
                    print(file_prefix,LQ_score,'Left_handed')
                if LQ_score >=-40 and LQ_score <=40: 
                    print(file_prefix,LQ_score,'Ambidextrous')
                if LQ_score > 40:
                    print(file_prefix,LQ_score,'Right_handed')
                subj_with_handedness.append([file_prefix,LQ_score])
            except ZeroDivisionError as e:
                print(file_prefix+ ' cant_divide_by_0_check_data')
# ,((sum(right)-sum(left))/(sum(right)+sum(left))))*100)
# IF(B20<-40,"Left handed",IF(AND(B20>=-40,B20<=40),"Ambidextrous",IF(B20>40,"Right handed"))))

Handedness_scoring_template_9054_1 52.94117647058824 Right_handed
MET_Handedness_9128_1 80.0 Right_handed
MET_handedness_9322_1 100.0 Right_handed
MET_handedness_9867_1 40.0 Ambidextrous
MET_handedness_9052_1 88.23529411764706 Right_handed
MET_Handedness_9405_1 68.42105263157895 Right_handed
MET_Handedness_9092_1 66.66666666666666 Right_handed
Handedness_scoring_9316_1 100.0 Right_handed
MET_handedness_9156_1 42.857142857142854 Right_handed
MET_handedness_9101_1 86.66666666666667 Right_handed
MET_Handedness_9194_1 90.0 Right_handed
MET_handedness_9113_1 100.0 Right_handed
MET_Handedness_9054_1 52.94117647058824 Right_handed
MET_Handedness_9302_1 68.42105263157895 Right_handed
MET_Handedness_9228_1 90.0 Right_handed
MET_Handedness_9181_1 78.94736842105263 Right_handed
MET_handedness_9501_1 88.88888888888889 Right_handed
MET_handedness_9188_1 89.47368421052632 Right_handed
MET_Handedness_9197_1 70.0 Right_handed
MET_Handedness_9168_1 100.0 Right_handed
MET_handedness_9856_1 90.0 Right_ha

#### functions used or imports + xlsx --> csv file creation

In [6]:
## function to use
def process_files_in_directory(directory_path, search_string=None):
    column_names = []
    os.chdir(directory_path)

    for subj_data_file in os.listdir(directory_path):
        if search_string and search_string not in subj_data_file:
            continue 

        file_prefix = subj_data_file.split('.')[0]
        visit = subj_data_file.split('_')[-1]
        
        print(subj_data_file, visit)
        
        try:
            subjfile = pd.ExcelFile(subj_data_file, engine='openpyxl')
            sheet_names = subjfile.sheet_names
        except Exception as e:
            print(f"Error reading {subj_data_file}: {e}")
            continue
        
        for sheet_name in sheet_names:
            print(sheet_name)
            if 'ormula' in sheet_name:
                import_sheet = sheet_name
                try:
                    subj_df = pd.read_excel(subj_data_file, header=[0], dtype=str, sheet_name=import_sheet)
                    column_names.append(subj_df.columns.tolist())
                except Exception as e:
                    print(f"Error reading sheet {sheet_name} in {subj_data_file}: {e}")
                    continue
                
                if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
                    visit = subj_data_file.split('_')[-1].split('.')[0]
                else:
                    visit = subj_data_file.split('_')[-1].split('.')[0]

                subj_df.fillna(value="", axis=0, inplace=True)
                subj_df["file_name"] = file_prefix

                subj_df.to_csv(f"{file_prefix}_import.csv", index=False)

    return column_names



## redcap
def combine_data_for_rcupload(all_assess_dirs_path, column_mappings, str_from_measure_fname,project_name, output_file):

    all_dfs = [] 

    for filename in os.listdir(all_assess_dirs_path):
        # if filename.endswith('import.csv') and str_from_measure_fname in filename and (filename.startswith(project_name) or project_name in filename):
        if filename.endswith('import.csv') and str_from_measure_fname in filename and (filename.startswith(project_name)):

            print(f"Processing {filename}")
            print(filename)
            csv_file_path = os.path.join(all_assess_dirs_path, filename)
            
            df = pd.read_csv(csv_file_path,dtype=str)
            print(df.columns)

            df.rename(columns=column_mappings, inplace=True)
                
            try:
                if 'Visit' in df.columns:
                    if project_name=='ASD_MET':
                        df[["redcap_event_name"]]="visit"+df['Visit']+'_arm_2'
                    else:
                        df[["redcap_event_name"]]="visit"+df['Visit']+'_arm_1'
                if 'Time' in df.columns:
                    if project_name=='ASD_MET':
                        df[["redcap_event_name"]]="visit"+df['Time']+'_arm_2'
                    else:
                        df[["redcap_event_name"]]="visit"+df['Time']+'_arm_1'
                print(df["redcap_event_name"])
                df[["record_id"]]=df.PID
                df = df[df.PID != "Assessor to complete"]
                cols = ['record_id', 'redcap_event_name','file_name'] + [col for col in df if col not in ['record_id', 'redcap_event_name','file_name']]
                df = df[cols]
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                df = pd.DataFrame(columns=column_mappings.values())

            all_dfs.append(df)


    combined_df = pd.concat(all_dfs, ignore_index=True)
    output_f_name=project_name+'_'+output_file
    combined_df.to_csv(os.path.join(all_assess_dirs_path,output_f_name), index=False)
    print(f"Combined CSV saved as {output_f_name}")



###### use below to make import from xlsx

In [7]:
# all_assess_dirs='/Users/daelsaid/slack_downloads/update_ados_adi/assess/csv-;'%%bash
# all_assess_dirs='/Users/daelsaid/scratch/np/behavioral_data-selected_2/all_neuropsych_pc-all'
# all_assess_dirs='/Users/daelsaid/scratch/np/behavioral_data-selected/all_from_main_box'
# all_assess_dirs='/Users/daelsaid/scratch/np/scsnl_neuropsych_pc_data-selected/all_assessments_virtual'
# all_assess_dirs='/Users/daelsaid/scratch/np/20211118/behavioral_data-selected_main_final/all_assessments_box_final'
# all_assess_dirs='/Volumes/de_encryptd/adi_from_neuropsych_pc/'
column_names=[]
os.chdir(all_assess_dirs)

# for data in 'wasi' 'gearys' 'wiat3' 'cbo' 'atl' 
for subj_data_file in os.listdir(all_assess_dirs):
    file_prefix=subj_data_file.split('.')[0]
    visit=subj_data_file.split('_')[-1].split('.')[0]
    if 'MET_' in subj_data_file:
        print(subj_data_file,visit)
        try:
            subjfile=pd.ExcelFile(subj_data_file,engine='openpyxl')
            s=subjfile.sheet_names
        except:
            continue
        for i,x in enumerate(s):
            # if 'Scorer1' in s[i]:
            #     import_sheet=s[i]
            #     subj_df=pd.read_excel(subj_data_file,header=[0],skiprows=None,dtype=str,sheet_name=import_sheet)
            #     column_names.append(subj_df.columns.tolist())
            #     if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #     else:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #         # visit=print(subj_df['Time'].tolist()[0])
            #     subj_df.fillna(value="",axis=0,inplace=True)            
            #     # Write the file_prefix value to the empty cell
            #     col_number=int(len(subj_df.columns))
            #     subj_df["file_name"]=file_prefix
            #     # Write the file_prefix value to the empty cell
            #     # subj_df.iloc[1, col_number+1] = file_prefix
            #     subj_df.to_csv(file_prefix + '_sc1.csv', index=False)
            # if 'Scorer2' in s[i]:
            #     import_sheet=s[i]
            #     subj_df=pd.read_excel(subj_data_file,header=[0],skiprows=None,dtype=str,sheet_name=import_sheet)
            #     column_names.append(subj_df.columns.tolist())
            #     if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #     else:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #         # visit=print(subj_df['Time'].tolist()[0])
            #     subj_df.fillna(value="",axis=0,inplace=True)            
            #     # Write the file_prefix value to the empty cell
            #     col_number=int(len(subj_df.columns))
            #     subj_df["file_name"]=file_prefix
            #     # Write the file_prefix value to the empty cell
            #     # subj_df.iloc[1, col_number+1] = file_prefix
            #     subj_df.to_csv(file_prefix + '_sc2.csv', index=False)
            # if 'Assessor' in s[i]:
            #     import_sheet=s[i]
            #     subj_df=pd.read_excel(subj_data_file,header=[0],skiprows=None,dtype=str,sheet_name=import_sheet)
            #     column_names.append(subj_df.columns.tolist())
            #     if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #     else:
            #         visit=subj_data_file.split('_')[-1].split('.')[0]
            #         # visit=print(subj_df['Time'].tolist()[0])
            #     subj_df.fillna(value="",axis=0,inplace=True)            
            #     # Write the file_prefix value to the empty cell
            #     col_number=int(len(subj_df.columns))
            #     subj_df["file_name"]=file_prefix
            #     # Write the file_prefix value to the empty cell
            #     # subj_df.iloc[1, col_number+1] = file_prefix
                #     subj_df.to_csv(file_prefix + '_asx.csv', index=False)
            if 'ormula' in s[i]:
                
                import_sheet=s[i]
                subj_df=pd.read_excel(subj_data_file,header=[0],skiprows=None,dtype=str,sheet_name=import_sheet)
                column_names.append(subj_df.columns.tolist())
                if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
                    visit=subj_data_file.split('_')[-1].split('.')[0]
                else:
                    visit=subj_data_file.split('_')[-1].split('.')[0]
                    # visit=print(subj_df['Time'].tolist()[0])
                subj_df.fillna(value="",axis=0,inplace=True)            
                # Write the file_prefix value to the empty cell
                col_number=int(len(subj_df.columns))
                subj_df["file_name"]=file_prefix
                # subj_df.iloc[1, col_number+1] = file_prefix
                subj_df.to_csv(file_prefix + '_import.csv', index=False)

MET_WISC_V_9072_1.xlsx 1
MET_neuropsych_CBO_9302_1.xlsx 1
MET_Pre_Strategy_Assessment_9072_1.xlsx 1
MET_WJ_III_9302_1.xlsx 1
FINAL_IMPORT_MET_Strategy_Assessment_9316_1.xlsx 1
MET_WASI_II_9072_1.xlsx 1


/Users/daelsaid/tools/miniconda3/env/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


MET_ATM_9128_2.xlsx 2
MET_MARS(SEMA)_9072_1.xlsx 1
DKEFS_MET_9054_1.xlsx 1
MET_WIAT_III_9228_1.xlsx 1
MET_Learning_Curve_9476_1.xlsx 1
MET_Pre_Strategy_Assessment_9156_1.xlsx 1
MET_WISC_V_9156_1.xlsx 1
MET_ATL_9501_1.xlsx 1
MET_WIAT_III_9181_1.xlsx 1
MET_WASI_II_9156_1.xlsx 1
MET_Geary's_9857_2.xlsx 2
MET_ATL_9188_1.xlsx 1
MET_SPT_CBO_9476_1.xlsx 1
MET_MARS(SEMA)_9156_1.xlsx 1
MET_WIAT_III_9347_1.xlsx 1
MET_Subtraction_9501_1.xlsx 1
MET_MARS(SEMA)_9228_1.xlsx 1
MET_WISC_V_9347_1.xlsx 1
MET_Handedness_9128_1.xlsx 1
MET_WJ_III_9054_1.xlsx 1
MET_Pre_Strategy_Assessment_9347_1.xlsx 1
MET_Learning_Curve_9128_1.xlsx 1
MET_MARS(SEMA)_9181_1.xlsx 1
MET_DKEFS_9194_1.xlsx 1
MET_WASI_II_9347_1.xlsx 1
MET_WISC_V_9228_1.xlsx 1
MET_WASI_II_9181_1.xlsx 1
MET_Pre_Strategy_Assessment_9228_1.xlsx 1
DKEFS_MET_9302_1.xlsx 1
MET_MARS(SEMA)_9347_1.xlsx 1
MET_Subtraction_9188_1.xlsx 1
MET_WIAT_III_9156_1.xlsx 1
MET_Pre_Strategy_Assessment_9181_1.xlsx 1
MET_WASI_II_9228_1.xlsx 1
MET_WISC_V_9181_1.xlsx 1
MET_M

#####  Read file w/ dictionary mapping

In [8]:
data_dictionary_path = '/Users/daelsaid/scratch/NeuroPsych/07_15_2022_16_05_51/NeuroPsych/all_assessments_box_final/tomake_into_df.txt'

data_dict_df = pd.read_csv(data_dictionary_path, delimiter='\t')

print(data_dict_df)

# Remove rows with any NaN values in 'Column A', 'Column B', or 'measure'
data_dict_df.dropna(subset=['A', 'B', 'measure'], inplace=True)
measure_dictionaries = {}

for measure, group in data_dict_df.groupby('measure'):
    measure_dictionaries[measure] = dict(zip(group['A'], group['B']))

print(measure_dictionaries['DKEFS'])


    measure                                                  A  \
0       NaN                                                PID   
1       NaN                                               Time   
2     DKEFS                                    Assessment Date   
3     DKEFS                                             Age_Yr   
4     DKEFS                                            Age_Mon   
..      ...                                                ...   
787     CBO                   Number of hours for assessment.2   
788     CBO  Child's ability to focus attention for long pe...   
789     CBO  Likelihood that the child will be able to sit ...   
790     CBO  Parent interactions (How they treated assessor...   
791     CBO           Any other things we should be aware of.2   

                                                     B  Unnamed: 3  \
0                                            record_id         NaN   
1                                    redcap_event_name         NaN 

### Generate sc1, asx, sc2, import formula tab to csv

##### FINAL strat assessment xlsx --> csv assessment specific 

In [7]:
# all_assess_dirs='/Users/daelsaid/scratch/Neuropsych/2023_11_06/updated_scores/sa'
os.chdir(all_assess_dirs)

os.listdir()


['python_tab_files',
 'FINAL_IMPORT_MET_Strategy_Assessment_9476_3.xlsx',
 'MET_Pre_Strategy_Assessment_9072_1.xlsx',
 'MET_Pre_Strategy_Assessment_9494_1.xlsx',
 'MET_Pre_Strategy_Assessment_9172_1.xlsx',
 'MET_Pre_Strategy_Assessment_9011_1.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9362_2.xlsx',
 'MET_Post_Strategy_Assessment_9304_2.xlsx',
 'strategy_asess_9092_Time_11_W_2020_08_31.csv',
 'FINAL_IMPORT_MET_Strategy_Assessment_9316_1.xlsx',
 'MET_Pre_Strategy_Assessment_9483_2.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9182_3.xlsx',
 'MET_FollowUp_Strategy_Assessment_9116_3.xlsx',
 'MET_Post_Strategy_Assessment_9316_2.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9370_2.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9304_1.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9407_3.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9203_3.xlsx',
 'FINAL_IMPORT_MET_Strategy_Assessment_9409_2.xlsx',
 'MET_FollowUp_Strategy_Assessment_9120_3.xlsx',
 'MET_Pre_Strategy_Assessment_9156_1.xlsx',
 

###### generate final import of strategy assessment summary as csv

In [8]:
column_names = process_files_in_directory(os.path.join(all_assess_dirs), search_string='FINAL')


FINAL_IMPORT_MET_Strategy_Assessment_9476_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9362_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9316_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9182_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9370_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9304_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9407_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9203_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9409_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9092_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9225_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9085_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9463_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9237_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9337_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9197_1.xlsx 1.xlsx

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Import
FINAL_IMPORT_MET_Strategy_Assessment_9489_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9856_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9372_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9206_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9237_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9265_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9434_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9337_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9092_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9409_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9203_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9407_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9370_3.xlsx 3.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9322_1.xlsx 1.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9182_2.xlsx 2.xlsx
Import
FINAL_IMPORT_MET_Strategy_Assessment_9099_1.xlsx

###### create output (final SA if above doesnt work)

In [6]:
# all_assess_dirs='/Users/daelsaid/slack_downloads/update_ados_adi/assess/csv-;'%%bash
# all_assess_dirs='/Users/daelsaid/scratch/np/behavioral_data-selected_2/all_neuropsych_pc-all'
# all_assess_dirs='/Users/daelsaid/scratch/np/behavioral_data-selected/all_from_main_box'
# all_assess_dirs='/Users/daelsaid/scratch/np/scsnl_neuropsych_pc_data-selected/all_assessments_virtual'
# all_assess_dirs='/Users/daelsaid/scratch/np/20211118/behavioral_data-selected_main_final/all_assessments_box_final'
# all_assess_dirs='/Volumes/de_encryptd/adi_from_neuropsych_pc/'
column_names=[]
os.chdir(all_assess_dirs)

# for data in 'wasi' 'gearys' 'wiat3' 'cbo' 'atl' 
for subj_data_file in os.listdir(all_assess_dirs):
    file_prefix=subj_data_file.split('.')[0]
    visit=subj_data_file.split('_')[-1]
    if 'FINAL_' in subj_data_file:
        print(subj_data_file,visit)
        try:
            subjfile=pd.ExcelFile(subj_data_file,engine='openpyxl')
            s=subjfile.sheet_names
        except:
            continue
        for i,x in enumerate(s):
            if 'Import' in s[i]:
                
                import_sheet=s[i]
                subj_df=pd.read_excel(subj_data_file,header=[0],skiprows=None,dtype=str,sheet_name=import_sheet)
                column_names.append(subj_df.columns.tolist())
                if 'andedness' in file_prefix or 'neuropsych_CBO' in file_prefix:
                    visit=subj_data_file.split('_')[-1].split('.')[0]
                else:
                    visit=subj_data_file.split('_')[-1].split('.')[0]
                    # visit=print(subj_df['Time'].tolist()[0])
                subj_df.fillna(value="",axis=0,inplace=True)            
                # Write the file_prefix value to the empty cell
                col_number=int(len(subj_df.columns))
                subj_df["file_name"]=file_prefix
                # subj_df.iloc[1, col_number+1] = file_prefix
                subj_df.to_csv(file_prefix + '_import.csv', index=False)

FINAL_IMPORT_MET_Strategy_Assessment_9476_3.xlsx 3.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9052_3_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9362_2.xlsx 2.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9316_1.xlsx 1.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9225_1_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9501_1_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9182_3.xlsx 3.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9489_1_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9434_2_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9352_1_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9370_2.xlsx 2.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9288_1_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9390_3_import.csv import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9304_1.xlsx 1.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9407_3.xlsx 3.xlsx
FINAL_IMPORT_MET_Strategy_Assessment_9203_3.xlsx 3.xlsx
FINAL_IMPORT_MET_Strateg

#### function to combine data

In [9]:
def combine_data_for_rcupload(all_assess_dirs_path, column_mappings, str_from_measure_fname,project_name, output_file):

    all_dfs = [] 

    for filename in os.listdir(all_assess_dirs_path):
        # if filename.endswith('import.csv') and str_from_measure_fname in filename and (filename.startswith(project_name) or project_name in filename):
        if filename.endswith('import.csv') and str_from_measure_fname in filename and (filename.startswith(project_name)):

            print(f"Processing {filename}")
            print(filename)
            csv_file_path = os.path.join(all_assess_dirs_path, filename)
            
            df = pd.read_csv(csv_file_path,dtype=str)
            print(df.columns)

            df.rename(columns=column_mappings, inplace=True)
                
            try:
                if 'Visit' in df.columns:
                    if project_name=='ASD_MET':
                        df[["redcap_event_name"]]="visit"+df['Visit']+'_arm_2'
                    else:
                        df[["redcap_event_name"]]="visit"+df['Visit']+'_arm_1'
                if 'Time' in df.columns:
                    if project_name=='ASD_MET':
                        df[["redcap_event_name"]]="visit"+df['Time']+'_arm_2'
                    else:
                        df[["redcap_event_name"]]="visit"+df['Time']+'_arm_1'
                print(df["redcap_event_name"])
                df[["record_id"]]=df.PID
                df = df[df.PID != "Assessor to complete"]
                cols = ['record_id', 'redcap_event_name','file_name'] + [col for col in df if col not in ['record_id', 'redcap_event_name','file_name']]
                df = df[cols]
            except Exception as e:
                print(f"Error processing {filename}: {e}")
                df = pd.DataFrame(columns=column_mappings.values())
 
            all_dfs.append(df)


    combined_df = pd.concat(all_dfs, ignore_index=True)
    output_f_name=project_name+'_'+output_file
    combined_df.to_csv(os.path.join(all_assess_dirs_path,output_f_name), index=False)
    print(f"Combined CSV saved as {output_f_name}")

In [7]:
def replace_q_resp_values(df, swapped_dict):
    # Find columns that end with 'q1_resp'
    q_resp_columns = [col for col in df.columns if col.endswith('_resp')]
    print(q_resp_columns)
    # Replace the values in each 'q1_resp' column using the swapped_dict
    for col in q_resp_columns:
        df.replace({col:swapped_dict},inplace=True)
        # = df[col].map(swapped_dict)
    
    return df



#### Generate Redcap upload csv

###### Strategy final import

In [13]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['final_sa']
search_string = 'Strategy'
output_file_name = search_string+'_rc_mapping_w_completion.csv'

# Call the function
for project in ['FINAL']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)


Processing FINAL_IMPORT_MET_Strategy_Assessment_9052_3_import.csv
FINAL_IMPORT_MET_Strategy_Assessment_9052_3_import.csv
Index(['PID', 'Visit', 'Assessment Date 1', 'Assessment Date 2', 'Acc_1',
       'Acc_2', 'Acc_3', 'Acc_4', 'CorrectRT_1', 'CorrectRT_2', 'CorrectRT_3',
       'CorrectRT_4', 'Counting_1', 'Counting_2', 'Counting_3', 'Counting_4',
       'Decomposition_1', 'Decomposition_2', 'Decomposition_3',
       'Decomposition_4', 'Retrieval_1', 'Retrieval_2', 'Retrieval_3',
       'Retrieval_4', 'ListPriming_1', 'ListPriming_2', 'ListPriming_3',
       'ListPriming_4', 'Other_1', 'Other_2', 'Other_3', 'Other_4',
       'Trained_Acc', 'Trained_CorrectRT', 'Trained_Counting',
       'Trained_Decomp', 'Trained_Retrieval', 'Trained_ListPriming',
       'Trained_Other', 'Untrained_Acc', 'Untrained_CorrectRT',
       'Untrained_Counting', 'Untrained_Decomp', 'Untrained_Retrieval',
       'Untrained_ListPriming', 'Untrained_Other', 'Total_Acc',
       'Total_CorrectRT', 'Total_Countin

###### DKEFS

In [10]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['DKEFS']
search_string = 'DKEFS'
output_file_name = search_string+'_rc_mapping_w_completion.csv'

# Call the function

for project in ['MET','ASD_MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)


Processing MET_DKEFS_9322_1_import.csv
MET_DKEFS_9322_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Age_Yr', 'Age_Mon', 'Age_Day',
       '1_Raw', '1_Scaled', '2_Raw', '2_Scaled', '3_Raw', '3_Scaled', '4_Raw',
       '4_Scaled', '5_Raw', '5_Scaled', '2+3_Sum_Scaled',
       '2+3_Composite_Scaled', '4vs1_Scaled_Dif', '4vs1_Scaled_Contrast',
       '4vs2_Scaled_Dif', '4vs2_Scaled_Contrast', '4vs3_Scaled_Dif',
       '4vs3_Scaled_Contrast', '4vs2+3_Scaled_Dif', '4vs2+3_Scaled_Contrast',
       '4vs5_Scaled_Dif', '4vs5_Scaled_Contrast', '1_Om_Error_Raw',
       '1_Om_Error_Rank', '1_Com_Error_Raw', '1_Com_Error_Rank',
       '2_Seq_Error_Raw', '2_Seq_Error_Rank', '3_Seq_Error_Raw',
       '3_Seq_Error_Rank', '4_Seq_Error_Raw', '4_Seq_Error_Rank',
       '2_SetLoss_Error_Raw', '2_SetLoss_Error_Rank', '3_SetLoss_Error_Raw',
       '3_SetLoss_Error_Rank', '4_SetLoss_Error_Raw', '4_SetLoss_Error_Rank',
       '2_Time_Error_Raw', '2_Time_Error_Rank', '3_Time_Error_Raw',
       '3_Time_

ValueError: No objects to concatenate

###### Neuropsych CBO

In [11]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['CBO']
search_string = 'neuropsych_CBO'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)

Processing MET_neuropsych_CBO_9228_1_import.csv
MET_neuropsych_CBO_9228_1_import.csv
Index(['Date', 'Number of hours for assessment',
       'Child's ability to focus attention for long periods of time',
       'Likelihood that the child will be able to sit still for 2 hrs inside the scanner',
       'Parent interactions (How they treated assessor, the child, any other observations)',
       'Any other things we should be aware of', 'Date.1',
       'Number of hours for assessment.1',
       'Child's ability to focus attention for long periods of time.1',
       'Likelihood that the child will be able to sit still for 2 hrs inside the scanner.1',
       'Parent interactions (How they treated assessor, the child, any other observations).1',
       'Any other things we should be aware of.1', 'Date.2',
       'Number of hours for assessment.2',
       'Child's ability to focus attention for long periods of time.2',
       'Likelihood that the child will be able to sit still for 2 hrs insi

###### ATM

In [12]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['atm']
search_string = 'ATM'
output_file_name = search_string+'_rc_mapping.csv'


for project in ['MET']:
# for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)

# radio
# 0	Not at All
# 1	A little
# 2	Somewhat
# 3	Very
# 4	Very Very


Processing MET_ATM_9347_1_import.csv
MET_ATM_9347_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Q1_Resp', 'Q1_Raw', 'Q2_Resp',
       'Q2_Raw', 'Q3_Resp', 'Q3_Raw', 'Q4_Resp', 'Q4_Raw', 'Q5_Resp', 'Q5_Raw',
       'Q6_Resp', 'Q6_Raw', 'Science_Resp', 'Science_Raw', 'English_Resp',
       'English_Raw', 'Math_Resp', 'Math_Raw', 'Art_Resp', 'Art_Raw',
       'Gym_Resp', 'Gym_Raw', 'Social_Studies_Resp', 'Social_Studies_Raw',
       'Music_Resp', 'Music_Raw', 'Total_ATM', 'Affect_ATM', 'Self_Perp_ATM',
       'Other_GAA', 'Assessor', 'Scorer1', 'Notes_from_Template',
       'Version_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_ATM_9476_1_import.csv
MET_ATM_9476_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Q1_Resp', 'Q1_Raw', 'Q2_Resp',
       'Q2_Raw', 'Q3_Resp', 'Q3_Raw', 'Q4_Resp', 'Q4_Raw', 'Q5_Resp', 'Q5_Raw',
       'Q6_Resp', 'Q6_Raw', 'Science_Resp', 'Science_Raw', 'English_Resp',
       'E

###### gearys

In [13]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['geary']
search_string = 'Geary'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET']:
# for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



Processing MET_Geary's_9156_1_import.csv
MET_Geary's_9156_1_import.csv
Index(['PID', 'Time', 'Age_Yr', 'Age_Mon', 'Age_Day', 'Assessment_Date',
       'Set51_Hit_Total', 'Set51_Hit_Same', 'Set51_Hit_Different',
       'Set51_Miss_Total', 'Set51_Miss_Same', 'Set51_Miss_Different',
       'Set51_False_Alarm_Total', 'Set51_False_Alarm_Same',
       'Set51_False_Alarm_Different', 'Set51_Correct_Rejection_Total',
       'Set51_Correct_Rejection_Same', 'Set51_Correct_Rejection_Different',
       'Set51_Not_Attempted_Total', 'Set51_Not_Attempted_Same',
       'Set51_Not_Attempted_Different', 'Set51_Time', 'Set52_Hit_Total',
       'Set52_Hit_Arabic', 'Set52_Hit_Combo', 'Set52_Miss_Total',
       'Set52_Miss_Arabic', 'Set52_Miss_Combo', 'Set52_False_Alarm_Total',
       'Set52_False_Alarm_Arabic', 'Set52_False_Alarm_Combo',
       'Set52_Correct_Rejection_Total', 'Set52_Correct_Rejection_Arabic',
       'Set52_Correct_Rejection_Combo', 'Set52_Not_Attempted_Total',
       'Set52_Not_Attempted_A

###### WISC V

In [14]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['wiscv']
search_string = 'WISC_V'
output_file_name = search_string+'_rc_mapping.csv'



for project in ['MET']:
# for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_WISC_V_9054_1_import.csv
MET_WISC_V_9054_1_import.csv
Index(['PID', 'Time', 'Assessment Date', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'DS_Raw', 'DSf_Raw', 'DSb_Raw', 'DSs_Raw', 'LN_Raw', 'SSP_Raw',
       'SSPf_Raw', 'SSPb_Raw', 'DS_Std', 'DSf_Std', 'DSb_Std', 'DSs_Std',
       'LN_Std', 'SSP_Std', 'SSPf_Std', 'SSPb_Std', 'Assessor', 'Scorer1',
       'Scorer2', 'Error_Code', 'Notes_from_Template', 'Version', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_WISC_V_9228_1_import.csv
MET_WISC_V_9228_1_import.csv
Index(['PID', 'Time', 'Assessment Date', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'DS_Raw', 'DSf_Raw', 'DSb_Raw', 'DSs_Raw', 'LN_Raw', 'SSP_Raw',
       'SSPf_Raw', 'SSPb_Raw', 'DS_Std', 'DSf_Std', 'DSb_Std', 'DSs_Std',
       'LN_Std', 'S

###### WIAT III

In [16]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['WIAT']
search_string = 'WIAT_III'
output_file_name = search_string+'_rc_mapping.csv'



for project in ['MET']:
# for project in ['MET','ASD_MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_WIAT_III_9054_1_import.csv
MET_WIAT_III_9054_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'Reading Comprehension_Raw', 'Math Problem Solving_Raw',
       'Word Reading_Raw', 'Pseudoword Decoding_Raw',
       'Numerical Operations_Raw', 'Math Fluency - Addition_Raw',
       'Math Fluency - Subtraction_Raw', 'Math Fluency - Multiplication_Raw',
       'Reading Comprehension_Std', 'Math Problem Solving_Std',
       'Word Reading_Std', 'Pseudoword Decoding_Std',
       'Numerical Operations_Std', 'Math Fluency - Addition_Std',
       'Math Fluency - Subtraction_Std', 'Math Fluency - Multiplication_Std',
       'Assessor', 'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template',
       'Version', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: r

###### WASI

In [24]:
all_assess_dirs_path = all_assess_dirs
column_name_mapping = measure_dictionaries['wasi']
search_string = 'WASI'
output_file_name = search_string+'_rc_mapping.csv'


# for project in ['ASD_MET','MET','AML']:
for project in ['MET']:   
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



Processing MET_WASI_II_9156_1_import.csv
MET_WASI_II_9156_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Age_Yr', 'Age_Mon', 'Age_Day', 'VIQ',
       'PIQ', 'FSIQ', 'Vocab_Raw', 'BlockD_Raw', 'Sim_Raw', 'MatrixR_Raw',
       'Vocab_Std', 'BlockD_Std', 'Sim_Std', 'MatrixR_Std', 'Assessor',
       'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_WASI_II_9857_1_import.csv
MET_WASI_II_9857_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Age_Yr', 'Age_Mon', 'Age_Day', 'VIQ',
       'PIQ', 'FSIQ', 'Vocab_Raw', 'BlockD_Raw', 'Sim_Raw', 'MatrixR_Raw',
       'Vocab_Std', 'BlockD_Std', 'Sim_Std', 'MatrixR_Std', 'Assessor',
       'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_WASI_II_9072_1_import.csv
MET_WASI_II_9072_1_import.csv
Index(['PID', '

###### ATL

In [17]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['atl']
search_string = 'ATL'
output_file_name = search_string+'_rc_mapping.csv'


for project in ['MET']:
# for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)
    
# atl={'0':'Very Very Disagree',
# '1':'Disagree',
# '2':'Somewhat Disagree',
# '3':'Somewhat Agree',
# '4':'Agree',
# '5':'Very Very Agree'}

# swapped_dict = {value: key for key, value in atl.items()}


dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_ATL_9322_1_import.csv
MET_ATL_9322_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Q1_Resp', 'Q1_Raw', 'Q2_Resp',
       'Q2_Raw', 'Q3_Resp', 'Q3_Raw', 'Q4_Resp', 'Q4_Raw', 'Q5_Resp', 'Q5_Raw',
       'Q6_Resp', 'Q6_Raw', 'Q7_Resp', 'Q7_Raw', 'Q8_Resp', 'Q8_Raw',
       'Q9_Resp', 'Q9_Raw', 'Q10_Resp', 'Q10_Raw', 'Q11_Resp', 'Q11_Raw',
       'Q12_Resp', 'Q12_Raw', 'Total Growth', 'Total Fixed', 'Total ATL',
       'Total Math', 'Total Reading', 'Total Smartness', 'Assessor', 'Scorer1',
       'Notes_from_Template', 'Version_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_ATL_9197_1_import.csv
MET_ATL_9197_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Q1_Resp', 'Q1_Raw', 'Q2_Resp',
       'Q2_Raw', 'Q3_Resp', 'Q3_Ra

###### SEMA

In [25]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['marsema']
search_string = 'SEMA'
output_file_name = search_string+'_rc_mapping.csv'


# for project in ['ASD_MET','MET','AML']:
for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)
    df=pd.read_csv(output_file_name,dtype=str)


    q_resp_columns = [col for col in df.columns if col.endswith('_resp')]
    print(f"Response columns in project {project_name}: {q_resp_columns}")

    for col in q_resp_columns:
        print(f"Unique values in column {col} before replacement:", df[col].unique())
        
        df[col] = df[col].map(swapped_dict).fillna(df[col])
        
        print(f"Unique values in column {col} after replacement:", df[col].unique())


    q_resp_columns = [col for col in df.columns if col.endswith('_resp')]
    print(q_resp_columns)
    for col in q_resp_columns:
        print(col)
        df.replace({col:swapped_dict},inplace=True)
        # = df[col].map(swapped_dict)

dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_MARS(SEMA)_9347_1_import.csv
MET_MARS(SEMA)_9347_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'P1_Resp', 'P1_Raw', 'Q1_Resp',
       'Q1_Raw', 'Q2_Resp', 'Q2_Raw', 'Q3_Resp', 'Q3_Raw', 'Q4_Resp', 'Q4_Raw',
       'Q5_Resp', 'Q5_Raw', 'Q6_Resp', 'Q6_Raw', 'Q7_Resp', 'Q7_Raw',
       'Q8_Resp', 'Q8_Raw', 'Q9_Resp', 'Q9_Raw', 'Q10_Resp', 'Q10_Raw',
       'P2_Resp', 'P2_Raw', 'Q11_Resp', 'Q11_Raw', 'Q12_Resp', 'Q12_Raw',
       'Q13_Resp', 'Q13_Raw', 'Q14_Resp', 'Q14_Raw', 'Q15_Resp', 'Q15_Raw',
       'Q16_Resp', 'Q16_Raw', 'Q17_Resp', 'Q17_Raw', 'Q18_Resp', 'Q18_Raw',
       'Q19_Resp', 'Q19_Raw', 'Q20_Resp', 'Q20_Raw', 'Sub1_Raw', 'Sub2_Raw',
       'Total_Raw', 'Assessor', 'Scorer1', 'Notes_from_Template',
       'Version_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name:

FileNotFoundError: [Errno 2] No such file or directory: 'SEMA_rc_mapping.csv'

In [77]:
sema={'1':'A little nervous',
'2':'Not nervous at all',
'3':'Somewhat nervous',
'4':'Very nervous',
'5':'Very very nervous',
'nan':'',
'check':'check'
}

swapped_sema = {value: key for key, value in sema.items()}

def remap_responses(column):
    if column.endswith('_resp'):
        return df[column].map(swapped_sema)
    else:
        return df[column]


###### WJ III

In [18]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['wj']
search_string = 'WJ'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET']:
# for project in ['MET','ASD_MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_WJ_III_import.csv
MET_WJ_III_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Grade', 'WJ_calculated grade', 'Sex',
       'Version', 'Calculation_Raw', 'Calculation_Std', 'MathFluency_Raw',
       'MathFluency_Std', 'MathFluency_Time', 'AppliedReason_Raw',
       'AppliedReason_Std', 'Assessor', 'Scorer1', 'Scorer2',
       'Scorer1_checked', 'Error_Code', 'Notes_from_Template', 'file_name'],
      dtype='object')
0    visit3_arm_1
Name: redcap_event_name, dtype: object
Processing MET_WJ_III_9156_1_import.csv
MET_WJ_III_9156_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Grade', 'WJ_calculated grade', 'Sex',
       'Version', 'Calculation_Raw', 'Calculation_Std', 'MathFluency_Raw',
       'MathFluency_Std', 'MathFluency_Time', 'AppliedReason_Raw',
       'AppliedReason_Std', 'Assessor', '

###### Subtraction

In [19]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['sub']
search_string = 'Subtraction'
output_file_name = search_string+'_rc_mapping.csv'

# for project in ['ASD_MET','MET','AML']:
for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_Subtraction_9168_2_import.csv
MET_Subtraction_9168_2_import.csv
Index(['PID', 'Time', 'Grade', 'Version_Pool', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'Assessment_Date', 'Raw_Score', 'Assessment_Time', 'Assessor',
       'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template',
       'Version_Template', 'file_name'],
      dtype='object')
0    visit2_arm_1
Name: redcap_event_name, dtype: object
Processing MET_Subtraction_9128_2_import.csv
MET_Subtraction_9128_2_import.csv
Index(['PID', 'Time', 'Grade', 'Version_Pool', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'Assessment_Date', 'Raw_Score', 'Assessment_Time', 'Assessor',
       'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template',
       'Version_Template', 'file_name'],
      dtype='object')
0    visit2_arm_1
Name: redcap_event_name, dtype: object

In [ ]:
np2_apts_aml=['WJ_IV','DKEFS','Geary','WISC']
for np apt in np2_apts_aml:
    if subj_id.split('.')[-1].split('_')[-1] == '1' and any(file_prefix).contains(np) and df.pid=:
        df['redcap_event_name']='np2_yr1_arm_1':
    

###### LC

In [20]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['lc']
search_string = 'Learning_Curve'
output_file_name = search_string+'_rc_mapping.csv'

# for project in ['ASD_MET','MET','AML']:
for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_Learning_Curve_9156_1_import.csv
MET_Learning_Curve_9156_1_import.csv
Index(['PID', 'Time', 'Grade', 'Version_Pool', 'Age_Yr_0', 'Age_Mon_0',
       'Age_Day_0', 'Assessment_Date_0', 'Week_0', 'Time_0',
       'Assessment_Date_1', 'Week_1', 'Time_1', 'Assessment_Date_2', 'Week_2',
       'Time_2', 'Assessment_Date_3', 'Week_3', 'Time_3', 'Assessment_Date_4',
       'Week_4', 'Time_4', 'Assessor', 'Scorer1', 'Scorer2', 'Error_Code',
       'Notes_from_Template', 'Version_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_Learning_Curve_9856_1_import.csv
MET_Learning_Curve_9856_1_import.csv
Index(['PID', 'Time', 'Grade', 'Version_Pool', 'Age_Yr_0', 'Age_Mon_0',
       'Age_Day_0', 'Assessment_Date_0', 'Week_0', 'Time_0',
       'Assessm

###### VNT

In [21]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['vnt']
search_string = 'VNT_Tutoring'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_VNT_Tutoring_9316_1_import.csv
MET_VNT_Tutoring_9316_1_import.csv
Index(['PID', 'Time', 'Grade', 'Version_Pool', 'Age_Yr_1', 'Age_Mon_1',
       'Age_Day_1', 'Assessment_Date_1', 'Production_Verbal_1',
       'Production_Spatial_1', 'Production_Total_1', 'Planet_Hunt_Verbal_1',
       'Planet_Hunt_Spatial_1', 'Planet_Hunt_Total_1', 'Planet_Hop_Verbal_1',
       'Planet_Hop_Spatial_1', 'Planet_Hop_Total_1', 'Number_Line_Verbal_1',
       'Number_Line_Spatial_1', 'Number_Line_Total_1', 'Assessment_Date_2',
       'Production_Verbal_2', 'Production_Spatial_1.1', 'Production_Total_2',
       'Planet_Hunt_Verbal_2', 'Planet_Hunt_Spatial_2', 'Planet_Hunt_Total_2',
       'Planet_Hop_Verbal_2', 'Planet_Hop_Spatial_2', 'Planet_Hop_Total_2',
       'Number_Line_Verbal_2', 'Number_Line_Spatial_2', 'Number_Line_

###### VNT CBO

In [22]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['vnt_cbo']
search_string = 'VNT_CBO'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_VNT_CBO_9052_1_import.csv
MET_VNT_CBO_9052_1_import.csv
Index(['PID', 'Visit', 'Version', 'Assessment_Date_1', 'Warm_Up_1',
       'Planet_Hunt_1', 'Planet_Hop_1', 'Number_Line_1', 'General_1',
       'Assessment_Date_2', 'Warm_Up_2', 'Planet_Hunt_2', 'Planet_Hop_2',
       'Number_Line_2', 'General_2', 'Assessment_Date_3', 'Warm_Up_3',
       'Planet_Hunt_3', 'Planet_Hop_3', 'Number_Line_3', 'General_3',
       'Assessment_Date_4', 'Warm_Up_4', 'Planet_Hunt_4', 'Planet_Hop_4',
       'Number_Line_4', 'General_4', 'Assessor', 'Scorer1', 'Version_Template',
       'file_name'],
      dtype='object')
0    visitcheck_arm_1
Name: redcap_event_name, dtype: object
Processing MET_VNT_CBO_9101_1_import.csv
MET_VNT_CBO_9101_1_import.csv
Index(['PID', 'Visit', 'Version', 'Assessment_Date_1', 'Warm_Up_1',
      

###### SPT tutoring

In [23]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['SPT']
search_string = 'SPT_Tutoring'
output_file_name = search_string+'_rc_mapping.csv'

for project in ['MET','ASD_MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_SPT_Tutoring_9188_1_import.csv
MET_SPT_Tutoring_9188_1_import.csv
Index(['PID', 'Visit', 'Grade', 'Version_Pool', 'Age_Yr_1', 'Age_Mon_1',
       'Age_Day_1', 'Assessment_Date_1', 'Warm_Up_1', 'Space_Hunt_1',
       'Bingo_1', 'Speeded_Production_1R1', 'Time_1R1',
       'Speeded_Production_1R2', 'Time_1R2', 'Speeded_Production_1R3',
       'Time_1R3', 'Assessment_Date_2', 'Warm_Up_2', 'Space_Hunt_2', 'Bingo_2',
       'Speeded_Production_2R1', 'Time_2R1', 'Speeded_Production_2R2',
       'Time_2R2', 'Speeded_Production_2R3', 'Time_2R3', 'Assessment_Date_3',
       'Warm_Up_3', 'Space_Hunt_3', 'Bingo_3', 'Speeded_Production_3R1',
       'Time_3R1', 'Speeded_Production_3R2', 'Time_3R2',
       'Speeded_Production_3R3', 'Time_3R3', 'Assessment_Date_4', 'Warm_Up_4',
       'Space_Hunt_4', 'Bingo_4', 'Spe

ValueError: No objects to concatenate

###### SPT CBO

In [24]:
all_assess_dirs_path = all_assess_dirs
print(measure_dictionaries.keys())
column_name_mapping = measure_dictionaries['SPT_CBO']
search_string = 'SPT_CBO'
output_file_name = search_string+'_rc_mapping.csv'

# for project in ['ASD_MET','MET','AML']:
for project in ['MET','ASD_MET']:
    project_name=project
    combine_data_for_rcupload(all_assess_dirs_path, column_name_mapping, search_string,project_name, output_file_name)



dict_keys(['CBO', 'DKEFS', 'SPT', 'SPT_CBO', 'VNT', 'WIAT', 'atl', 'atm', 'final_sa', 'geary', 'lc', 'marsema', 'marssema', 'sub', 'vnt', 'vnt_cbo', 'wasi', 'wiscv', 'wj'])
Processing MET_SPT_CBO_9188_1_import.csv
MET_SPT_CBO_9188_1_import.csv
Index(['PID', 'Visit', 'Version', 'Assessment_Date_1', 'Warm_Up_1',
       'Space_Hunt_1', 'Bingo_1', 'Beat_Your_Time_1', 'General_1',
       'Assessment_Date_2', 'Warm_Up_2', 'Space_Hunt_2', 'Bingo_2',
       'Beat_Your_Time_2', 'General_2', 'Assessment_Date_3', 'Warm_Up_3',
       'Space_Hunt_3', 'Bingo_3', 'Beat_Your_Time_3', 'General_3',
       'Assessment_Date_4', 'Warm_Up_4', 'Space_Hunt_4', 'Bingo_4',
       'Beat_Your_Time_4', 'General_4', 'Assessor', 'Scorer1',
       'Version_Template', 'file_name'],
      dtype='object')
0    visit1_arm_1
Name: redcap_event_name, dtype: object
Processing MET_SPT_CBO_9337_1_import.csv
MET_SPT_CBO_9337_1_import.csv
Index(['PID', 'Version', 'Assessment_Date_1', 'Warm_Up_1', 'Space_Hunt_1',
       'Bingo_1

ValueError: No objects to concatenate

#### TESTS
##### DKEFS

#### WJ check  calc prob 1 for first graders

In [123]:
# Set the data directory to a local path and change the working directory to it
all_assess_dirs='/Volumes/de_encryptd/np/2024_05_29/de_met_asdmet_wj'
all_assess_dirs=os.path.join(all_assess_dirs,'wj')


# all_assess_dirs='/Users/daelsaid/scratch/Neuropsych/2023_11_06/updated_scores/'
#/Users/daelsaid/scratch/NeuroPsych/all_assessments_box_final/'
# /Users/daelsaid/scratch/np/behavioral_data-selected_2/all_neuropsych_pc-all'
# /Users/daelsaid/scratch/np/behavioral_data-selected/all_from_main_box
os.chdir(all_assess_dirs)
print(os.getcwd())

# os.listdir()


# Create an empty dataframe to store the age group information
scores=pd.DataFrame()
for subj_data_file in os.listdir(all_assess_dirs):
    subtest_scores_asx=pd.DataFrame()
    subtest_score_sc1=pd.DataFrame()
    subtest_score_sc2=pd.DataFrame()
    print(subj_data_file)
    if 'WJ' in subj_data_file and '_import.csv' not in subj_data_file and 'MET' in subj_data_file and 'rc_mapping' not in subj_data_file:
        file_prefix=subj_data_file.split('.')[0]
        visit=file_prefix.split('_')[-1]
        print(subj_data_file,visit)
        # try:
        subjfile = pd.ExcelFile(subj_data_file, engine='openpyxl')
        # subjfile=pd.ExcelFile(subj_data_file,engine='openpyxl')
        s=subjfile.sheet_names
        # except:
        #     continue
        for i,x in enumerate(s):
            if 'ssessor' in s[i]:
                sheet=s[i]
                subj_df = pd.read_excel(subj_data_file,header=[0], dtype=str, sheet_name=sheet, engine='openpyxl')
                subj=subj_df.iloc[2:3, [1]].values[0]
                year=subj_df.iloc[3:4,[4]].values[0]
                val_in_temp1=subj_df.iloc[53:54,[0]].values[0]
                if not (val_in_temp1 == 'Total Raw Score'):
                    score=subj_df.iloc[64:65, [1]].values[0]
                    print(score)
                else:
                    score=subj_df.iloc[53:54, [1]].values[0]

                # display(subj_df.iloc[0:67, [0,1]])
                subtest_scores_asx['pid'] = subj
                subtest_scores_asx['visit']=visit
                subtest_scores_asx['grade']=year
                subtest_scores_asx[["asx_score"]]=score
                # display(subtest_scores_asx)
            if 'orer1' in s[i]:
                sheet=s[i]
                subj_df = pd.read_excel(subj_data_file,header=[0], dtype=str, sheet_name=sheet, engine='openpyxl')
                subj=subj_df.iloc[2:3, [1]].values[0]
                year=subj_df.iloc[3:4,[4]].values[0]
                val_in_temp1=subj_df.iloc[53:54,[0]].values[0]
                if not (val_in_temp1 == 'Total Raw Score'):
                    score=subj_df.iloc[64:65, [1]].values[0]
                    print(score)
                else:
                    score=subj_df.iloc[53:54, [1]].values[0]
                # print(score)
                # display(subj_df.iloc[0:67, [0,1]])

                # subtest_score_sc1['pid'] = subj
                # subtest_score_sc1['visit']=visit
                # subtest_score_sc1['grade']=year
                subtest_scores_asx[["sc1"]]=score
                # display(subtest_score_sc1)
                # display(subj_df.head(120))
            if 'orer2' in s[i]:
                sheet=s[i]
                subj_df = pd.read_excel(subj_data_file,header=[0], dtype=str, sheet_name=sheet, engine='openpyxl')
                subj=subj_df.iloc[2:3, [1]].values[0]
                year=subj_df.iloc[3:4,[4]].values[0]
                val_in_temp1=subj_df.iloc[53:54,[0]].values[0]
                if not (val_in_temp1 == 'Total Raw Score'):
                    score=subj_df.iloc[64:65, [1]].values[0]
                    print(score)
                else:
                    score=subj_df.iloc[53:54, [1]].values[0]
                # print(score)
                # display(subj_df.iloc[0:67, [0,1]])

                # subtest_score_sc2['pid'] = subj
                # subtest_score_sc2['visit']=visit
                # subtest_score_sc2['grade']=year
                subtest_scores_asx[["sc2"]]=score
                # display(subtest_score_sc2)
        subject_scores = pd.concat([subtest_scores_asx, subtest_score_sc1, subtest_score_sc2], ignore_index=True)
        scores = scores.append(subject_scores, ignore_index=True)
display(scores)
scores.to_csv('wj_total_score_for_calculation_all_tabs.csv')

/Volumes/de_encryptd/np/2024_05_29/de_met_asdmet_wj/wj
MET_WJ_III_9430_1.xlsx
MET_WJ_III_9430_1.xlsx 1
MET_WJ_III_9376_2.xlsx
MET_WJ_III_9376_2.xlsx 2


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


['20']
['17']
['20']
MET_WJ_III_9470_3.xlsx
MET_WJ_III_9470_3.xlsx 3
MET_WJ_III_9210_1.xlsx
MET_WJ_III_9210_1.xlsx 1
MET_WJ_III_9326_1.xlsx
MET_WJ_III_9326_1.xlsx 1
['16']
['19']
['19']
MET_WJ_III_9191_1.xlsx
MET_WJ_III_9191_1.xlsx 1
['11']
['3']
['11']
MET_WJ_III_9403_2.xlsx
MET_WJ_III_9403_2.xlsx 2
['22']
['22']
['22']
MET_WJ_III_9352_2.xlsx
MET_WJ_III_9352_2.xlsx 2
['16']
['16']
['16']
MET_WJ_III_9472_2.xlsx
MET_WJ_III_9472_2.xlsx 2
MET_WJ_III_9194_2.xlsx
MET_WJ_III_9194_2.xlsx 2
['18']
['18']
['18']
MET_WJ_III_9257_1.xlsx
MET_WJ_III_9257_1.xlsx 1
[nan]
[nan]
['0']
MET_WJ_III_9183_1.xlsx
MET_WJ_III_9183_1.xlsx 1
['17']
['17']
['16']
MET_WJ_III_9334_1.xlsx
MET_WJ_III_9334_1.xlsx 1
[nan]
[nan]
['0']
MET_WJ_III_9029_1.xlsx
MET_WJ_III_9029_1.xlsx 1
[nan]
[nan]
['0']
MET_WJ_III_9120_2.xlsx
MET_WJ_III_9120_2.xlsx 2
MET_WJ_III_9297_2.xlsx
MET_WJ_III_9297_2.xlsx 2
MET_WJ_III_9054_1.xlsx
MET_WJ_III_9054_1.xlsx 1
MET_WJ_III_9037_1.xlsx
MET_WJ_III_9037_1.xlsx 1
[nan]
[nan]
['0']
MET_WJ_III_928

,pid,visit,grade,asx_score,sc1,sc2
0,9430,1,3,16,16,16
1,9376,2,4,20,17,20
2,9470,3,3,29,29,29
3,9210,1,3,21,21,21
4,9326,1,3,16,19,19
...,...,...,...,...,...,...
276,NaN,1,NaN,NaN,NaN,0
277,9472,1,3,15,15,15
278,9194,1,3,14,14,14
279,9183,2,4,12,12,11


In [129]:
all_dfs = []

for filename in os.listdir(all_assess_dirs):
    if filename.endswith('import.csv') and 'WIAT' in filename:
        print(filename)
        csv_file_path = os.path.join(all_assess_dirs, filename)
        df = pd.read_csv(csv_file_path,dtype=str)
        print(df.columns)
        new_df=df.rename(columns=measure_dictionaries['DKEFS']) 
        print(new_df)
        # new_df.to_csv(os.path.join(all_assess_dirs,filename.split('.')[0]+'dkefs_rc_mapping.csv'), index=False)
        all_dfs.append(new_df)
        print(df['file_name'].split('_')[-1])


combined_df = pd.concat(all_dfs, ignore_index=True)
# combined_df.to_csv(os.path.join(all_assess_dirs, 'dkefs.csv'), index=False)


ASD_SL_WIAT_IV_12007_1_import.csv
Index(['PID', 'Time', 'Assessment_Date', 'Age_Yr', 'Age_Mon', 'Age_Day',
       'Reading Comprehension_Raw', 'Math Problem Solving_Raw',
       'Word Reading_Raw', 'Pseudoword Decoding_Raw',
       'Numerical Operations_Raw', 'Math Fluency - Addition_Raw',
       'Math Fluency - Subtraction_Raw', 'Math Fluency - Multiplication_Raw',
       'Reading Comprehension_Std', 'Math Problem Solving_Std',
       'Word Reading_Std', 'Pseudoword Decoding_Std',
       'Numerical Operations_Std', 'Math Fluency - Addition_Std',
       'Math Fluency - Subtraction_Std', 'Math Fluency - Multiplication_Std',
       'Assessor', 'Scorer1', 'Scorer2', 'Error_Code', 'Notes_from_Template',
       'Version', 'file_name'],
      dtype='object')
                    PID                  Time  \
0  Assessor to complete  Assessor to complete   

                                     Assessment_Date  \
0  Scorer1 to complete/Scorer1 to complete/Scorer...   

  fsid0103met_dkefs_0004_

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
# def combine_data_for_rcupload(all_assess_dirs_path, column_mappings, str_from_measure_fname, project_name, output_file):
#     all_dfs = []  

#     for filename in os.listdir(all_assess_dirs_path):
#         if filename.endswith('import.csv') and str_from_measure_fname in filename and filename.startswith(project_name):
#             print(f"Processing {filename}")
#             try:
#                 csv_file_path = os.path.join(all_assess_dirs_path, filename)
#                 df = pd.read_csv(csv_file_path, dtype=str)
#                 df.rename(columns=column_mappings, inplace=True)
#                 df["redcap_event_name"] = "visit" + df['Time'] + '_arm_1'
#                 df["record_id"] = df.PID
#             except Exception as e:
#                 print(f"Error processing {filename}: {e}")
#                 df = pd.DataFrame(columns=column_mappings.values())

#             all_dfs.append(df)

#     combined_df = pd.concat(all_dfs, ignore_index=True)
#     output_f_name = project_name + '_' + output_file
#     combined_df.to_csv(os.path.join(all_assess_dirs_path, output_f_name), index=False)
#     print(f"Combined CSV saved as {output_f_name}")



In [ ]:
# pip install xlrd
# import xlrd

# # Specify the path to the Excel file

# # Provide the password for the protected sheet
# password = 'stanford123'

# # Open the workbook using xlrd
# book = xlrd.open_workbook(file_contents=open(excel_file, 'rb').read(), on_demand=True, formatting_info=True, password=password)

# # Select the sheet you want to read
# sheet_name = 'sheet_name'

# # Read the sheet data into a DataFrame
# df = pd.read_excel(book, sheet_name=sheet_name)

# # Perform operations on the DataFrame
# print(df.head())

# # for subj_data_file in os.listdir(all_assess_dirs):
# #     file_prefix=subj_data_file.split('.')[0]
# #     print(file_prefix)
# #     try:    
# #         subj_df1=pd.read_excel(subj_data_file,header=[0],sheet_name=None,skiprows=None,dtype=str)
# #         keys=subj_df1.keys()
# #         k=[k for k in keys if 'port' in k]
# #         print(k)
# #         subj_df=pd.read_excel(subj_data_file,sheet_name=k[0],header=[0],skiprows=None,dtype=str)
# #         print(subj_df.head())
# #         visit=subj_df['Time'].tolist()[0]
# #         print(visit)
# #     except:
# #         continue

In [ ]:

# def create_column_mapping(file_path):
#     column_mapping = {}
#     with open(file_path, 'r') as file:
#         lines = file.readlines()
#         for line in lines:
#             parts = line.strip().split('\t')  
#             if len(parts) >= 2 and parts[1]:  
#                 column_mapping[parts[0]] = parts[1]  # Column A is the key, Column B is the value
#     return column_mapping

# file_path = '/Users/daelsaid/scratch/NeuroPsych/07_15_2022_16_05_51/NeuroPsych/all_assessments_box_final/to_make_into_df.txt'

# # Generate the column mapping
# column_name_mapping = create_column_mapping(file_path)

# # Output the result
# print(column_name_mapping)


In [ ]:
# with open('/Users/daelsaid/Downloads/filenames.txt', 'r') as f:
#     content = f.read()

# # Use regex to split the filenames
# filenames = re.split(r'(?<=\.csv)(?=[A-Z])', content)

# # Loop through each file and group them by project based on the prefix
# for filename in filenames:
#     prefix = extract_prefix(filename)
#     if prefix:
#         csv_file_path = os.path.join(all_assess_dirs,adhd, filename)
        
#         # Read the CSV into a DataFrame
#         df = pd.read_csv(csv_file_path)
        
#         # Replace the column names using the provided dictionary
#         df.rename(columns=column_name_mapping, inplace=True)
        
#         # Append the DataFrame to the corresponding project list
#         project_dataframes[prefix].append(df)

# # Concatenate all DataFrames within each project and save them to new CSV files
# for prefix, dfs in project_dataframes.items():
#     if dfs:  # If there are DataFrames in the list
#         combined_df = pd.concat(dfs, ignore_index=True)
#         combined_df.to_csv(f'/path/to/save/{prefix}_combined.csv', index=False)


#### get ADOS columns

In [20]:
for idx,col in enumerate(column_names):
    print(col[idx])

PID
Visit ID
Assessor
Scorer 1
Scorer 2
Date of Assessment (Year)
Date of Assessment (Month)
Date of Assessment (Day)
Age (Years)
Age (Months)
Age (Days)
A-1. Non-echoed language
A-2. Speech Abnormalities 
A-3. Immediate Echolalia
A-4. Stereotyped Words
A-5. Offers Information
A-6. Asks Information
A-7. Report Events
A-8. Conversation
A-9. Gestures
B-1. Unusual Eye Contact
B-2. Facial expressions
B-3. Language Production
B-4. Shared Enjoyment
B-5. Empathy
B-6. Insight
B-7. Social Overtures
B-8. Social Response
B-9. Reciprocal Social Interaction
B-10. Rapport
C-1. Imagination/Creativity
D-1. Unusual Sensory Interest
D-2. Complex Mannerisms
D-3. Self-Injury
C-1. Imagination/Creativity
D-5. Compulsions/Rituals
E-1. Overactivity/Agitation
E-2. Tantrums/Aggression/Disruptive
E-3. Anxiety
Old. Communication total
Old. Reciprocal Social Interaction Total
Old. Communication + Social Interaction Total
Old. Imagination/Creativity Total
Old. Imagination/Creativity Total
Old. Diagnosis
New. Social

IndexError: list index out of range

In [10]:
print(set_of_cols)

[{'67. Unusual preoccupations - Ever', '33. Stereotyped utterances and delayed Echolalia - Ever', '91. Drawing Skill (unusually skilled use of perspective or creative approach) - Current', "87. Interviewer's judgment on age when developmental abnormalities probably first manifest (Code in months)", '49. Imaginative play with peers - Most Abnormal', 'Current Total C: Restricted, Repetitive, and Stereotyped Patterns of Behavior', '42. Pointing to express interest - Most Abnormal', 'Diagnostic Total D: Abnormality of Development Evident at or Before 36 Months', '64. Group play with peers - Most Abnormal', '5. Age when first walked unaided (in months)', '39. Verbal Rituals - Current', '22. Motor skills', '73. Abnormal idiosyncratic negative response to specific sensory stimuli - Ever', '10. Age of first phrases (if ever used) - in months', 'Date of Interview (Year)', '41. Current communicative speech - Ever', '68. Circumscribed interests - Ever', '14. Loss of communicative intent', '75. Re